# 计算素数个数

我们做一个小实验：计算小于给定正整数 $N$ 的素数的个数。我们用一个函数 `is_prime` 来判断某个正整数 $n$ 是不是素数，是素数则返回 1，不是则返回 0。这只要遍历检查从 2 到 $\sqrt{n}$ 之间是否有整数能够整除 $n$ 即可。然后将小于 $n$ 的全部整数依次代入此函数并统计结果。

In [1]:
import time

N = 1000000

def is_prime(n: int):
    result = True
    for k in range(2, int(n**0.5) + 1):
        if n % k == 0:
            result = False
            break
    return result

def count_primes(n: int) -> int:
    count = 0
    for k in range(2, n):
        if is_prime(k):
            count += 1

    return count

start = time.perf_counter()
print(f"Number of primes: {count_primes(N)}")
print(f"time elapsed: {time.perf_counter() - start}/s")

Number of primes: 78498
time elapsed: 4.4212623/s


然后我们引入 `taichi`，分别给 `is_prime` 函数和 `count_primes` 函数加上一个装饰器，再比较下耗时：

In [2]:
import taichi as ti
ti.init(arch=ti.cpu)

@ti.func
def is_prime(n: int):
    result = True
    for k in range(2, int(n**0.5) + 1):
        if n % k == 0:
            result = False
            break
    return result

@ti.kernel
def count_primes(n: int) -> int:
    count = 0
    for k in range(2, n):
        if is_prime(k):
            count += 1

    return count

start = time.perf_counter()
print(f"Number of primes: {count_primes(N)}")
print(f"time elapsed: {time.perf_counter() - start}/s")

[Taichi] version 0.8.3, llvm 10.0.0, commit 021af5d2, win, python 3.8.8
[Taichi] Starting on arch=x64
Number of primes: 78498
time elapsed: 0.10576790000000003/s


$N$ 等于一百万时 Taichi 的加速效果也许看起来没有那么惊人，这是因为在 binder 的服务器上 CPU 的多线程并行是受到限制的。我们建议你把此 notebook 下载到本地，然后试试把 $N$ 改成一千万，看看加速效果如何？

In [4]:
import random
def generate_data():
    points = []
    for i in range(10000):
        points.append([random.random(), random.random()])
    return points
def find_nn(src, tar):
    res_points = []
    for q in src:
        min_dist = 100000.0
        best_index = 0
        for p in tar:
            dist = (p[0] - q[0])**2 + (p[1] - q[1])**2
            if dist < min_dist:
                min_dist = dist
                best_point = p
    return best_point
src = generate_data()
tar = generate_data()

start = time.perf_counter()
find_nn(src, tar)
print(f"time elapsed: {time.perf_counter() - start}/s")

time elapsed: 46.07493049999994/s


In [32]:
import taichi as ti
ti.init(arch=ti.gpu)
N = 1000000
src = ti.Vector.field(2, dtype=ti.f32, shape=[N])
tar = ti.Vector.field(2, dtype=ti.f32, shape=[N])
res = ti.Vector.field(2, dtype=ti.f32, shape=[N])
@ti.kernel
def init():
    for i in src:
        src[i] = ti.Vector([ti.random(), ti.random()])
    for i in tar:
        tar[i] = ti.Vector([ti.random(), ti.random()])

@ti.func
def find_nn_one(p):
    min_dist = 100000.0
    best_point = ti.Vector([ti.random(), ti.random()])
    for i in range(N):
        q = tar[i]
        diff = p - q
        dist = diff.norm()
        if dist < min_dist:
            min_dist = dist
            best_point = p
    return best_point

@ti.kernel
def find_nn():
    res_points = []
    for i in src:
        p = src[i]
        res[i] = find_nn_one(p)
init()
start = time.perf_counter()
find_nn()
print(f"time elapsed: {time.perf_counter() - start}/s")

[Taichi] Starting on arch=cuda
time elapsed: 0.15500000000065484/s


In [9]:
position = ti.Vector.field(2, dtype=ti.f32, shape=[10])
print(position)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
